# Imports

In [ ]:
# Only required if using Google Colab and Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [1]:
import os
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

# OSM context - add heights based on near neighbors

In [ ]:
DATA_FOLDER = # ADD DATA FOLDER PATH, e.g. '/content/drive/MyDrive/Project_Name/data/'

In [3]:
# Load the version of splitlines_df that is classified and has WWRs assigned,
# which contains detected WWRs at street-facing fronts from the deep learning model
# and estimated WWRs at non-street-facing sides and rears from the classification algorithm
# each row is one line segment from the GIS polygons, split at vertices.

splitlines_df = pd.read_excel(DATA_FOLDER+'classified_splitlines_with_WWRs.xlsx')
splitlines_df

,OBJECTID,osm_id,osm_id_final,Start_Lon,Start_Lat,End_Lon,End_Lat,Center_Lon,Center_Lat,LINE_LENGTH,LINE_BEARING,Line_Bearing_Normal,osm_polygon_length,osm_polygon_area,CIT_AR,Era,Main_Class,Final_Class,GFA,in_prep_ds,lower_window_area,upper_window_area,lower_wall_area,upper_wall_area,garage_area,lower_wall_area_NET,total_window_area,total_wall_area,WWR_lower,WWR_upper,WWR_total,have_image,is_front,is_attached_side,is_connecting_mid_segment,is_angled_front,is_front_final,is_side,is_rear,status,WWR_final,have_min_one_detection,within_thresh
0,21,83471240,83471240_1,7.708351,45.095895,7.708318,45.095332,7.708334,45.095614,62.646436,182.411618,92.411618,149.503891,758.362410,184907,4,AB,R4AB,5308.536873,True,16930.345056,84877.205472,90693.376505,419706.168332,0.0,90693.376505,101807.550527,510399.544837,0.186677,0.202230,0.199466,True,True,False,False,False,True,False,False,front,0.199466,True,True
1,22,83471240,83471240_2,7.708318,45.095332,7.708164,45.095337,7.708241,45.095335,12.109366,272.427715,182.427715,149.503891,758.362410,184907,4,AB,R4AB,5308.536873,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,True,False,side,0.050000,True,True
2,23,83471240,83471240_3,7.708164,45.095337,7.708198,45.095900,7.708181,45.095619,62.646619,2.418707,272.418707,149.503891,758.362410,184907,4,AB,R4AB,5308.536873,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,True,rear,0.202230,True,True
3,24,83471240,83471240_4,7.708198,45.095900,7.708351,45.095895,7.708274,45.095898,12.101470,92.428606,2.428606,149.503891,758.362410,184907,4,AB,R4AB,5308.536873,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,True,False,side,0.050000,True,True
4,25,83471241,83471241_1,7.709266,45.096186,7.708926,45.095812,7.709096,45.095999,49.416307,212.704064,122.704064,122.521160,585.214298,184893,4,AB,R4AB,4096.500085,True,2320.703730,20356.036889,31650.655384,131043.144541,0.0,31650.655384,22676.740619,162693.799925,0.073322,0.155338,0.139383,True,True,False,False,False,True,False,False,front,0.139383,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5630,9759,1214584770,1214584770_4,7.697510,45.106176,7.697719,45.106113,7.697615,45.106144,17.931609,113.090262,23.090262,65.066177,257.733721,123075,5,AB,R5AB,1288.668604,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,True,rear,0.190556,True,True
5631,9768,1214584772,1214584772_1,7.698279,45.106267,7.698200,45.106145,7.698239,45.106206,14.912131,204.734596,114.734596,61.426195,234.187350,122005,4,AB,R4AB,936.749400,True,0.000000,0.000000,89068.562937,0.000000,0.0,89068.562937,0.000000,89068.562937,0.000000,NaN,0.000000,True,True,False,False,False,True,False,False,front,0.000000,True,True
5632,9769,1214584772,1214584772_2,7.698200,45.106145,7.698022,45.106201,7.698111,45.106173,15.290303,294.134786,204.134786,61.426195,234.187350,122005,4,AB,R4AB,936.749400,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,True,False,side,0.050000,True,True
5633,9770,1214584772,1214584772_3,7.698022,45.106201,7.698073,45.106321,7.698048,45.106261,13.924715,16.666161,286.666161,61.426195,234.187350,122005,4,AB,R4AB,936.749400,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,True,False,side,0.050000,True,True


In [4]:
# Load detected_stories, which contains the detected no. stories and height
# from images, to fill gaps in OSM data.
# This will be joined to splitlines_df so that this data can be exported and then
# imported into GIS, so that the SHP files can be exported for UBEM purposes.

detected_stories = pd.read_excel(DATA_FOLDER+'OSM_detected_height_and_stories.xlsx')
detected_stories

,osm_id,no_stories_muni,bldg_height_muni,stories_detected,height_detected,diff_height,diff_stories
0,83471241,7,23.299999,7,23.367086,0.067087,0
1,108229725,10,34.410000,10,35.778717,1.368717,0
2,108229726,7,26.330000,7,25.320982,-1.009018,0
3,114796212,6,21.610001,5,16.448215,-5.161785,-1
4,117354756,6,20.750000,5,18.302934,-2.447066,-1
...,...,...,...,...,...,...,...
302,1214584655,2,8.200000,2,7.857911,-0.342089,0
303,1214584659,2,6.970000,2,6.978477,0.008477,0
304,1214584661,4,13.120000,4,14.092752,0.972753,0
305,1214584741,1,4.300000,1,3.681132,-0.618868,0


In [5]:
# Merge splitlines_df with detected_stories, then limit splitlines_df to where there are heights/stories detected only.

splitlines_df = pd.merge(splitlines_df, detected_stories[['osm_id', 'height_detected', 'stories_detected']], on='osm_id', how='left')
splitlines_df = splitlines_df.loc[pd.notna(splitlines_df['height_detected'])].reset_index(drop=True)
splitlines_df

,OBJECTID,osm_id,osm_id_final,Start_Lon,Start_Lat,End_Lon,End_Lat,Center_Lon,Center_Lat,LINE_LENGTH,LINE_BEARING,Line_Bearing_Normal,osm_polygon_length,osm_polygon_area,CIT_AR,Era,Main_Class,Final_Class,GFA,in_prep_ds,lower_window_area,upper_window_area,lower_wall_area,upper_wall_area,garage_area,lower_wall_area_NET,total_window_area,total_wall_area,WWR_lower,WWR_upper,WWR_total,have_image,is_front,is_attached_side,is_connecting_mid_segment,is_angled_front,is_front_final,is_side,is_rear,status,WWR_final,have_min_one_detection,within_thresh,height_detected,stories_detected
0,25,83471241,83471241_1,7.709266,45.096186,7.708926,45.095812,7.709096,45.095999,49.416307,212.704064,122.704064,122.521160,585.214298,184893,4,AB,R4AB,4096.500085,True,2320.703730,20356.036889,31650.655384,131043.144541,0.0,31650.655384,22676.740619,162693.799925,0.073322,0.155338,0.139383,True,True,False,False,False,True,False,False,front,0.139383,True,True,23.367086,7.0
1,26,83471241,83471241_2,7.708926,45.095812,7.708799,45.095869,7.708863,45.095841,11.845371,302.669919,212.669919,122.521160,585.214298,184893,4,AB,R4AB,4096.500085,True,0.000000,0.000000,92842.284812,0.000000,0.0,92842.284812,0.000000,92842.284812,0.000000,NaN,0.000000,True,True,False,False,False,True,False,False,front,0.000000,True,True,23.367086,7.0
2,27,83471241,83471241_3,7.708799,45.095869,7.709139,45.096243,7.708969,45.096056,49.420687,32.711571,302.711571,122.521160,585.214298,184893,4,AB,R4AB,4096.500085,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,True,False,side,0.050000,True,True,23.367086,7.0
3,28,83471241,83471241_4,7.709139,45.096243,7.709266,45.096186,7.709203,45.096214,11.838795,122.691267,32.691267,122.521160,585.214298,184893,4,AB,R4AB,4096.500085,True,0.000000,0.000000,105292.959237,0.000000,0.0,105292.959237,0.000000,105292.959237,0.000000,NaN,0.000000,True,True,False,False,False,True,False,False,front,0.000000,True,True,23.367086,7.0
4,117,108229725,108229725_1,7.704146,45.092445,7.704017,45.092373,7.704082,45.092409,12.943890,232.035603,142.035603,147.912029,776.389691,258957,5,AB,R5AB,7763.896914,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,True,False,side,0.050000,True,True,35.778717,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1714,9664,1214584741,1214584741_4,7.699318,45.108098,7.699450,45.108048,7.699384,45.108073,11.847176,118.417469,28.417469,38.020621,85.522292,123034,4,SF,R4SF,85.522292,True,3163.206561,0.000000,18480.836933,0.000000,0.0,18480.836933,3163.206561,18480.836933,0.171161,NaN,0.171161,True,True,False,False,False,True,False,False,front,0.171161,True,True,3.681132,1.0
1715,9768,1214584772,1214584772_1,7.698279,45.106267,7.698200,45.106145,7.698239,45.106206,14.912131,204.734596,114.734596,61.426195,234.187350,122005,4,AB,R4AB,936.749400,True,0.000000,0.000000,89068.562937,0.000000,0.0,89068.562937,0.000000,89068.562937,0.000000,NaN,0.000000,True,True,False,False,False,True,False,False,front,0.000000,True,True,15.798781,5.0
1716,9769,1214584772,1214584772_2,7.698200,45.106145,7.698022,45.106201,7.698111,45.106173,15.290303,294.134786,204.134786,61.426195,234.187350,122005,4,AB,R4AB,936.749400,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,True,False,side,0.050000,True,True,15.798781,5.0
1717,9770,1214584772,1214584772_3,7.698022,45.106201,7.698073,45.106321,7.698048,45.106261,13.924715,16.666161,286.666161,61.426195,234.187350,122005,4,AB,R4AB,936.749400,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,True,False,side,0.050000,True,True,15.798781,5.0


# Weight window and wall areas by orientation

In [6]:
# Line_Bearing_Normal is the line bearing of each segment, minus 90 degrees, converted to a value between 0 and 360.
# This conversion was done in a previous script and now 'Line_Bearing_Normal' is saved in splitlines_df.
# Here we convert Line_Bearing_Normal to an orientation of north/east/south/west.

splitlines_df['is_North'] = np.where((splitlines_df['Line_Bearing_Normal'] < 45) | (splitlines_df['Line_Bearing_Normal'] >= 315), True, False)
splitlines_df['is_East'] = np.where((splitlines_df['Line_Bearing_Normal'] < 135) & (splitlines_df['Line_Bearing_Normal'] >= 45), True, False)
splitlines_df['is_South'] = np.where((splitlines_df['Line_Bearing_Normal'] < 225) & (splitlines_df['Line_Bearing_Normal'] >= 135), True, False)
splitlines_df['is_West'] = np.where((splitlines_df['Line_Bearing_Normal'] < 315) & (splitlines_df['Line_Bearing_Normal'] >= 225), True, False)

In [7]:
# Function to determine WWR for each orientation (north, east, south, west), weighted by line length.
# If the segment is an attached side, it is weighted as zero, so that it is excluded from WWR calculations.

def weight_wwr_by_orientation(osm_id):
  """
  Function to format inputs for the subsequent NMS functions

  Arguments:
  osm_id                --  string; a unique building identifier.

  Returns:
  tmp_df                --  pd.DataFrame; dataframe with added columns for weighting each line segment based on orientation (north/east/south/west),
                            and calculated WWR values for each orientation, to be used for UBEM.
  """
  tmp_df = splitlines_df.loc[splitlines_df['osm_id'] == osm_id].copy()
  total_line_len_north = tmp_df.apply(lambda row: row['LINE_LENGTH'] if row['is_attached_side']!=True and row['is_North'] else 0, axis=1).sum()
  total_line_len_east = tmp_df.apply(lambda row: row['LINE_LENGTH'] if row['is_attached_side']!=True and row['is_East'] else 0, axis=1).sum()
  total_line_len_south = tmp_df.apply(lambda row: row['LINE_LENGTH'] if row['is_attached_side']!=True and row['is_South'] else 0, axis=1).sum()
  total_line_len_west = tmp_df.apply(lambda row: row['LINE_LENGTH'] if row['is_attached_side']!=True and row['is_West'] else 0, axis=1).sum()

  tmp_df.loc[:,'line_weight_north'] = tmp_df.apply(lambda row: row['LINE_LENGTH'] / total_line_len_north if row['is_attached_side']!=True and row['is_North'] else 0, axis=1)
  tmp_df.loc[:,'line_weight_east'] = tmp_df.apply(lambda row: row['LINE_LENGTH'] / total_line_len_east if row['is_attached_side']!=True and row['is_East'] else 0, axis=1)
  tmp_df.loc[:,'line_weight_south'] = tmp_df.apply(lambda row: row['LINE_LENGTH'] / total_line_len_south if row['is_attached_side']!=True and row['is_South'] else 0, axis=1)
  tmp_df.loc[:,'line_weight_west'] = tmp_df.apply(lambda row: row['LINE_LENGTH'] / total_line_len_west if row['is_attached_side']!=True and row['is_West'] else 0, axis=1)

  wwr_north = tmp_df['WWR_final'] * tmp_df['line_weight_north']
  wwr_east = tmp_df['WWR_final'] * tmp_df['line_weight_east']
  wwr_south = tmp_df['WWR_final'] * tmp_df['line_weight_south']
  wwr_west = tmp_df['WWR_final'] * tmp_df['line_weight_west']
  wwr_north = wwr_north.sum()
  wwr_east = wwr_east.sum()
  wwr_south = wwr_south.sum()
  wwr_west = wwr_west.sum()
  tmp_df['wwr_north'] = wwr_north
  tmp_df['wwr_east'] = wwr_east
  tmp_df['wwr_south'] = wwr_south
  tmp_df['wwr_west'] = wwr_west
  return tmp_df

In [8]:
# Test and debug one example.

idx = 100
osm_id = splitlines_df.loc[idx, 'osm_id']
print(osm_id)
weight_wwr_by_orientation(osm_id)

215457706


,OBJECTID,osm_id,osm_id_final,Start_Lon,Start_Lat,End_Lon,End_Lat,Center_Lon,Center_Lat,LINE_LENGTH,LINE_BEARING,Line_Bearing_Normal,osm_polygon_length,osm_polygon_area,CIT_AR,Era,Main_Class,Final_Class,GFA,in_prep_ds,lower_window_area,upper_window_area,lower_wall_area,upper_wall_area,garage_area,lower_wall_area_NET,total_window_area,total_wall_area,WWR_lower,WWR_upper,WWR_total,have_image,is_front,is_attached_side,is_connecting_mid_segment,is_angled_front,is_front_final,is_side,is_rear,status,WWR_final,have_min_one_detection,within_thresh,height_detected,stories_detected,is_North,is_East,is_South,is_West,line_weight_north,line_weight_east,line_weight_south,line_weight_west,wwr_north,wwr_east,wwr_south,wwr_west
98,786,215457706,215457706_1,7.706614,45.125763,7.706061,45.125606,7.706337,45.125684,46.845024,248.095281,158.095281,116.272734,528.814379,237714,4,MF,R4MF,1586.443138,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,True,rear,0.189112,True,True,10.66206,3.0,False,False,True,False,0.0,0.0,1.0,0.0,0.1869,0.05,0.189112,0.05
99,787,215457706,215457706_2,7.706061,45.125606,7.706007,45.125700,7.706034,45.125653,11.289268,338.089470,248.089470,116.272734,528.814379,237714,4,MF,R4MF,1586.443138,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,True,False,side,0.050000,True,True,10.66206,3.0,False,False,False,True,0.0,0.0,0.0,1.0,0.1869,0.05,0.189112,0.05
100,788,215457706,215457706_3,7.706007,45.125700,7.706560,45.125857,7.706284,45.125779,46.852203,68.098939,338.098939,116.272734,528.814379,237714,4,MF,R4MF,1586.443138,True,28438.798678,58433.640885,155818.190211,308989.599403,0.0,155818.190211,86872.439563,464807.789614,0.182513,0.189112,0.1869,True,True,False,False,False,True,False,False,front,0.186900,True,True,10.66206,3.0,True,False,False,False,1.0,0.0,0.0,0.0,0.1869,0.05,0.189112,0.05
101,789,215457706,215457706_4,7.706560,45.125857,7.706614,45.125763,7.706587,45.125810,11.286239,158.126179,68.126179,116.272734,528.814379,237714,4,MF,R4MF,1586.443138,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,True,False,side,0.050000,True,True,10.66206,3.0,False,True,False,False,0.0,1.0,0.0,0.0,0.1869,0.05,0.189112,0.05


In [9]:
# Run for loop on splitline, calling weight_wwr_by_orientation using one osm_id at a time.

df_lst = []
counter = 0
for osm_id in splitlines_df['osm_id'].unique():
  tmp_df = weight_wwr_by_orientation(osm_id)
  df_lst.append(tmp_df)
  counter +=1
  if counter % 100 == 0:
    print(counter)
print(counter)

wwr_orientation_df = pd.concat(df_lst, ignore_index=True)
wwr_orientation_df

100
200
300
307


,OBJECTID,osm_id,osm_id_final,Start_Lon,Start_Lat,End_Lon,End_Lat,Center_Lon,Center_Lat,LINE_LENGTH,LINE_BEARING,Line_Bearing_Normal,osm_polygon_length,osm_polygon_area,CIT_AR,Era,Main_Class,Final_Class,GFA,in_prep_ds,lower_window_area,upper_window_area,lower_wall_area,upper_wall_area,garage_area,lower_wall_area_NET,total_window_area,total_wall_area,WWR_lower,WWR_upper,WWR_total,have_image,is_front,is_attached_side,is_connecting_mid_segment,is_angled_front,is_front_final,is_side,is_rear,status,WWR_final,have_min_one_detection,within_thresh,height_detected,stories_detected,is_North,is_East,is_South,is_West,line_weight_north,line_weight_east,line_weight_south,line_weight_west,wwr_north,wwr_east,wwr_south,wwr_west
0,25,83471241,83471241_1,7.709266,45.096186,7.708926,45.095812,7.709096,45.095999,49.416307,212.704064,122.704064,122.521160,585.214298,184893,4,AB,R4AB,4096.500085,True,2320.703730,20356.036889,31650.655384,131043.144541,0.0,31650.655384,22676.740619,162693.799925,0.073322,0.155338,0.139383,True,True,False,False,False,True,False,False,front,0.139383,True,True,23.367086,7.0,False,True,False,False,0.0,1.0,0.000000,0.0,0.000000,0.139383,0.000000,0.050000
1,26,83471241,83471241_2,7.708926,45.095812,7.708799,45.095869,7.708863,45.095841,11.845371,302.669919,212.669919,122.521160,585.214298,184893,4,AB,R4AB,4096.500085,True,0.000000,0.000000,92842.284812,0.000000,0.0,92842.284812,0.000000,92842.284812,0.000000,NaN,0.000000,True,True,False,False,False,True,False,False,front,0.000000,True,True,23.367086,7.0,False,False,True,False,0.0,0.0,1.000000,0.0,0.000000,0.139383,0.000000,0.050000
2,27,83471241,83471241_3,7.708799,45.095869,7.709139,45.096243,7.708969,45.096056,49.420687,32.711571,302.711571,122.521160,585.214298,184893,4,AB,R4AB,4096.500085,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,True,False,side,0.050000,True,True,23.367086,7.0,False,False,False,True,0.0,0.0,0.000000,1.0,0.000000,0.139383,0.000000,0.050000
3,28,83471241,83471241_4,7.709139,45.096243,7.709266,45.096186,7.709203,45.096214,11.838795,122.691267,32.691267,122.521160,585.214298,184893,4,AB,R4AB,4096.500085,True,0.000000,0.000000,105292.959237,0.000000,0.0,105292.959237,0.000000,105292.959237,0.000000,NaN,0.000000,True,True,False,False,False,True,False,False,front,0.000000,True,True,23.367086,7.0,True,False,False,False,1.0,0.0,0.000000,0.0,0.000000,0.139383,0.000000,0.050000
4,117,108229725,108229725_1,7.704146,45.092445,7.704017,45.092373,7.704082,45.092409,12.943890,232.035603,142.035603,147.912029,776.389691,258957,5,AB,R5AB,7763.896914,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,True,False,side,0.050000,True,True,35.778717,10.0,False,False,True,False,0.0,0.0,0.436293,0.0,0.138491,0.130215,0.114628,0.210615
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1714,9664,1214584741,1214584741_4,7.699318,45.108098,7.699450,45.108048,7.699384,45.108073,11.847176,118.417469,28.417469,38.020621,85.522292,123034,4,SF,R4SF,85.522292,True,3163.206561,0.000000,18480.836933,0.000000,0.0,18480.836933,3163.206561,18480.836933,0.171161,NaN,0.171161,True,True,False,False,False,True,False,False,front,0.171161,True,True,3.681132,1.0,True,False,False,False,1.0,0.0,0.000000,0.0,0.171161,0.000000,0.171161,0.000000
1715,9768,1214584772,1214584772_1,7.698279,45.106267,7.698200,45.106145,7.698239,45.106206,14.912131,204.734596,114.734596,61.426195,234.187350,122005,4,AB,R4AB,936.749400,True,0.000000,0.000000,89068.562937,0.000000,0.0,89068.562937,0.000000,89068.562937,0.000000,NaN,0.000000,True,True,False,False,False,True,False,False,front,0.000000,True,True,15.798781,5.0,False,True,False,False,0.0,1.0,0.000000,0.0,0.164865,0.000000,0.050000,0.050000
1716,9769,1214584772,1214584772_2,7.698200,45.106145,7.698022,45.1062

In [10]:
# Create grouped_df using groupby on splitlines_df['osm_id'], aggretate sum of each window_area_north, wall_area_north, window_area_south, wall_area_south, etc.
# This data is to be exported back to GIS to be incorporated into the Shapefiles used in the UBEM

grouped_df = wwr_orientation_df.groupby('osm_id').agg({
    'stories_detected':'first',
    'height_detected':'first',
    'wwr_north':'first',
    'wwr_east':'first',
    'wwr_south':'first',
    'wwr_west':'first'})
grouped_df

,stories_detected,height_detected,wwr_north,wwr_east,wwr_south,wwr_west
osm_id,,,,,,
83471241,7.0,23.367086,0.000000,0.139383,0.000000,0.050000
108229725,10.0,35.778717,0.138491,0.130215,0.114628,0.210615
108229726,7.0,25.320982,0.230725,0.062434,0.136023,0.230725
114796212,5.0,16.448215,0.115419,0.050000,0.195342,0.050000
117354756,5.0,18.302934,0.211145,0.000000,0.225441,0.000000
...,...,...,...,...,...,...
1214584655,2.0,7.857911,0.000000,0.178271,0.000000,0.177129
1214584659,2.0,6.978477,0.000000,0.182287,0.050000,0.163181
1214584661,4.0,14.092752,0.257985,0.201923,0.000000,0.257985


In [11]:
# For all WWR_North, WWR_South, WWR_East, WWR_West values, if value = 0, change value to 0.0000001,
# as WWR = 0 doesn't work in the UBEM.

grouped_df['wwr_north'] = np.where(grouped_df['wwr_north'] == 0, 0.0000001, grouped_df['wwr_north'])
grouped_df['wwr_south'] = np.where(grouped_df['wwr_south'] == 0, 0.0000001, grouped_df['wwr_south'])
grouped_df['wwr_east'] = np.where(grouped_df['wwr_east'] == 0, 0.0000001, grouped_df['wwr_east'])
grouped_df['wwr_west'] = np.where(grouped_df['wwr_west'] == 0, 0.0000001, grouped_df['wwr_west'])
grouped_df.reset_index(inplace=True)
grouped_df

,osm_id,stories_detected,height_detected,wwr_north,wwr_east,wwr_south,wwr_west
0,83471241,7.0,23.367086,1.000000e-07,1.393829e-01,1.000000e-07,5.000000e-02
1,108229725,10.0,35.778717,1.384906e-01,1.302153e-01,1.146281e-01,2.106154e-01
2,108229726,7.0,25.320982,2.307250e-01,6.243381e-02,1.360231e-01,2.307250e-01
3,114796212,5.0,16.448215,1.154187e-01,5.000000e-02,1.953424e-01,5.000000e-02
4,117354756,5.0,18.302934,2.111449e-01,1.000000e-07,2.254412e-01,1.000000e-07
...,...,...,...,...,...,...,...
302,1214584655,2.0,7.857911,1.000000e-07,1.782711e-01,1.000000e-07,1.771291e-01
303,1214584659,2.0,6.978477,1.000000e-07,1.822867e-01,5.000000e-02,1.631810e-01
304,1214584661,4.0,14.092752,2.579849e-01,2.019234e-01,1.000000e-07,2.579849e-01
305,1214584741,1.0,3.681132,1.711614e-01,1.000000e-07,1.711614e-01,1.000000e-07


# Next steps in GIS

In [ ]:
# Export grouped_df, to be re-imported into GIS

folder_name = # ADD FOLDER NAME
file_name = # ADD FILE NAME + '.xlsx'
grouped_df.to_excel(folder_name + file_name, index=False)

In [ ]:
'''
NEXT STEPS IN GIS:

1.  Import grouped_df and join using 'osm_id' to the GIS layer containing OSM polygons.
    --> now the building polygon layer will contain WWR on a per-orientation basis.
2.  Use Select by Attributes to select each desired building archetype.
3.  Export each archetype to Shapefile format.
4.  Also export context layers to Shapefile format.
5.  Import Shapefiles into UBEM and connect geometry and metadata to appropriate inputs.

'''